In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import openai
import random

Mounted at /content/drive


In [ ]:
paragraphs = []

with open("all_paragraphs.txt", 'r') as file:
    paragraphs = file.read().split('\n')
# Optionally, to remove any empty strings if there are extra newlines
paragraphs = [para for para in paragraphs if para]

In [ ]:
len(paragraphs)

2636

In [ ]:
from openai import OpenAI
# Set your OpenAI API key

client = OpenAI(api_key = YOUR_API_KEY)

def extract_region(paragraph):
    if paragraph.startswith('['):
        tag, _, content = paragraph.partition('] ')
        return tag.strip('[]'), content.strip()
    return None, paragraph

def generate_summary(paragraph, region):
 # Prepend regional context if available
    if region:
        regional_context = f"In the context of climate change in {region}, "
        prompt_content = regional_context + paragraph
    else:
        prompt_content = paragraph
    try:
        response = client.chat.completions.create(
          model="gpt-3.5-turbo-1106",
          messages=[
              {"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": f"Generate an informative and in-depth summary of the following. Avoid using introductory phrases like 'This paragraph discusses'. Generate\n\n{prompt_content}"}
          ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

summarized_paragraphs = []

for paragraph in paragraphs:
    region, content = extract_region(paragraph)
    summary = generate_summary(content, region)
    print(f"Paragraph: {paragraph}\nGenerated Summary:\n{summary}\n")

    # Re-prepend the region to the summary if it was present
    if region:
        summarized_paragraph = f"[{region}] {summary}"
    else:
        summarized_paragraph = summary

    summarized_paragraphs.append(summarized_paragraph)

In [ ]:
summarized_file_path = "summarized_paragraphs.txt"

with open(summarized_file_path, 'w') as file:
    for summarized_paragraph in summarized_paragraphs:
        file.write(summarized_paragraph + '\n')